# Datacube Scene & Pixel Viewer

This notebook can be used to view the actual landsat scenes that urban change detection algorithms are based on.
The output of the urban change detection algorithm is a .img file for each landsat band, with each file having a band for each scene that was part of the analysis.
The change algorithm might have look at 500 scenes, and so the Red.img file, for example, would have 500 layers, each being the red band from each scene.

This notebook pulls the red, green and blue bands for a given scene, combines them, contrast stretches them into something resembling true colour, and plots them up. A slider allows different scenes to be displayed.
The results of the urban change detection algorithm are superimposed over the true colour scene. The transparency of these results can be changed by the use of a second slider.
Clicking on a pixel displays the spectral signature of that pixel, (across all available bands) to be displayed on the line chart below the image.

## TO DO LIST
<ul>
<li>Threshold lines
<li>Mouse Over TS for dates?
<li>Investigate threshold magnitudes from Peters vs this work
<li>Ask Peter about filtering/noise reduction
</ul>

### Imports

In [1]:
%matplotlib notebook
# %matplotlib inline
import os

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib
from matplotlib.pyplot import imshow

import gdal
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, FloatSlider
from IPython.display import display

from skimage import exposure
from scipy.signal import lfilter

import warnings

### Importing the data

In [63]:
# build a list of all files in the directory (ie the folder for that location)
location = 'mtbarker/'
files = os.listdir(location)

# build a list of all the NBAR*.img file names and which bands they represent
NBARfiles = []
bands = []
for file in files:
    if file[-4::] == '.img' and file[0:4] == 'NBAR':
        NBARfiles.append(file)
        bands.append(file.split('NBAR_')[1].split('.img')[0])

# open all the .img files with NBAR in the name, convert to numpy array, swap axes so order is (x, y, t)
# and save to dict
raw_data = {}
for i in range(len(NBARfiles)):
    raw_data[bands[i]] = gdal.Open(location + NBARfiles[i]).ReadAsArray().swapaxes(0,2)
num_scenes = len(raw_data['red'][0][0])

# build a list of all the dates represented by each band in the NBAR files
# reuse the list of NBAR file names, but this time access the .hdr file
in_dates = False
dates = []
for line in open(location + NBARfiles[0].split('.img')[0] + '.hdr'):
    if line[0] == '}':
        continue
    if in_dates:
        dates.append(line.split(',')[0].strip())
    if line[0:10] == 'band names':
        in_dates = True

# save list of satellite originated bands
sat_bands = bands.copy()

# add the yet to be calculated derivative bands to the overall bands list
bands += ['evi', 'ndvi', 'albedo', 'cloud_mask']

In [3]:
# define the size for the numpy array that will hold all the data for conversion into XArray
x = len(raw_data['red'])
y = len(raw_data['red'][0])
t = len(raw_data['red'][0][0])
n = len(bands)

# create an empty numpy array of the correct size
alldata = np.zeros((x, y, t, n), dtype=np.float32)

# populate the numpy array with the satellite data
# turn all no data NBAR values to NaNs
for i in range(len(sat_bands)):
    alldata[:,:,:,i] = raw_data[sat_bands[i]]
    alldata[:,:,:,i][alldata[:,:,:,i] == -999] = np.nan
    
# convert the numpy array into an xarray, with appropriate lables, and axes names
data = xr.DataArray(alldata, coords = {'x':range(x), 'y':range(y), 'date':dates, 'band':bands},
             dims=['x', 'y', 'date', 'band'])

In [4]:
# open the results of the change algorithm, and format for eay plotting
change = gdal.Open(location + 'change_time.img').ReadAsArray()

# remove the dates, so you have a mask for change or no change
change_flat_mask = change.copy()
change_flat_mask[change_flat_mask != 0] = 1
change_flat_mask[change_flat_mask == 0] = np.nan

### Creating the calculated "bands"

In [5]:
# calculate NDVI "band"
a = (data.loc[:,:,:,'nir'] - data.loc[:,:,:,'red'])
b = (data.loc[:,:,:,'nir'] + data.loc[:,:,:,'red'])
b_pos = b.where(b.values > 0)
data.loc[:,:,:,'ndvi'] = np.nan
data.loc[:,:,:,'ndvi'] = a/b_pos

# calculate EVI "band"
g  = 2.5
c1 = 6
c2 = 7.5
l  = 1
data.loc[:,:,:,'evi'] = g * ((a/10000) /
    ((data.loc[:,:,:,'nir']/10000) + (c1 * (data.loc[:,:,:,'red']/10000)) - (c2 * (data.loc[:,:,:,'blue']/10000)) + l))

# calculate average albedo "band"
for band in sat_bands:
    data.loc[:,:,:,'albedo'] += data.loc[:,:,:,band]
data.loc[:,:,:,'albedo'] = data.loc[:,:,:,'albedo'] / (6 * 1000)

# import cloudmask and add to xarray
cloudmask = gdal.Open(location + '/tsmask.img').ReadAsArray().swapaxes(0,2)
data.loc[:,:,:,'cloud_mask'] = cloudmask

/g/data/v10/public/modules/agdc-py3-env/20171016/envs/agdc/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in greater
  after removing the cwd from sys.path.


In [6]:
def gatherAndPrepTSData(xpos,ypos,ts_bands):
    # get data and apply cloud mask
    # xpos and ypos are lists at this point, the getTimeSeries function unpacks them accordingly

#     {'name':'evi_raw', 'band':'evi', 'cloud_mask':True, 'smoothed':True, 'RollingAve' = True, 'TSDiff':False}
    dfs = []
    cols = []
    for TS in ts_bands:
        df = getTimeSeries(xpos, ypos, TS['band'], apply_cloud_mask = TS['cloud_mask'])
        # drop rows with clouds masked
        df = df.dropna(axis = 0, how = 'all')

        # filter noise
        if TS['smoothed']:
            df = filterNoise(df)

        # calculate rolling average
        if TS['RollingAve']:
            df = calcRollingAvg(df)
            # drop rows with no rolling average values
            df = df.dropna(axis = 0, how = 'all')

        # calculate difference between TS value and a period into the future
        if TS['TSDiff']:
            df = calcYearDiff(df)
        
        dfs.append(df)
        cols.append(TS['name'])
    dfs = pd.concat(dfs, axis = 1)
    dfs.columns = cols
    return dfs

In [15]:
test = []
test.append({'name':'evi_diff', 'band':'evi', 'cloud_mask':True, 'smoothed':True, 'RollingAve':True, 'TSDiff':True})
test.append({'name':'evi_raw', 'band':'evi', 'cloud_mask':True, 'smoothed':True, 'RollingAve':True, 'TSDiff':False})
test.append({'name':'albedo', 'band':'evi', 'cloud_mask':True, 'smoothed':True, 'RollingAve':True, 'TSDiff':True})

gatherAndPrepTSData([20,40],[24,40],test)

,evi_diff,evi_raw,albedo
date,,,
1988-01-26,0.156211,0.078570,0.156211
1988-03-30,0.170366,0.086250,0.170366
1988-05-17,0.175283,0.093617,0.175283
1988-06-02,0.175932,0.101843,0.175932
1988-08-05,0.179598,0.110867,0.179598
1988-08-21,0.173141,0.120157,0.173141
1988-09-06,0.167657,0.130261,0.167657
1988-09-22,0.150171,0.150120,0.150171
1988-10-08,0.131964,0.170342,0.131964


In [7]:
def getTimeSeries(xpos, ypos, ts_bands, apply_cloud_mask = True):
    """
    This function retrives the time series data for a given pixel and band.
    It then applies the cloud/shadow mask to the results and to the dates for each scene.
    Finally, it returns the good pixels for each of the requested bands as a multi column pandas dataframe,
    with the date formatted as a datetime object and set as in the index
    
    Possible flaw with logic by only masking pixels when they are clouded, not entire study area if any or some
    is partially clouded. Need to explore this thought further
    """       
    if isinstance(xpos, list) and len(xpos) == 1:
        xpos.append(xpos[0] + 1)
    elif not isinstance(xpos, list) and is_number(xpos):
        xpos = [int(xpos), xpos + 1]
        
    if isinstance(ypos, list) and len(ypos) == 1:
        ypos.append(ypos[0] + 1)
    elif not isinstance(ypos, list) and is_number(ypos):
        ypos = [int(ypos), ypos + 1]
    
    # if looking for an area and not masking for cloud, do this
    if not apply_cloud_mask:
        ts_data = pd.DataFrame(data[xpos[0]:xpos[1], ypos[0]:ypos[1],:].sel(band = ts_bands).mean(dim=['x','y'], skipna = True).to_pandas())
        
    # if looking for an area and masking for cloud, do this
    else:
        ts_data = pd.DataFrame(data[xpos[0]:xpos[1], ypos[0]:ypos[1], :].sel(band = ts_bands).where(
            data[xpos[0]:xpos[1], ypos[0]:ypos[1], :].sel(band ='cloud_mask') == 0).mean(dim=['x','y'], skipna = True).to_pandas())
    
    ts_data.index = pd.to_datetime(ts_data.index) 
    ts_data.columns = [ts_bands]
    return ts_data

def define_TS_YAxisRange(data, DiffTS = False, n_stddevs = 0.5):
    """
    This function defines the y axis range for a timeseries plot of a given band (supplied as a pandas 
    single column dataframe or series, aka data. It calculates the standard deviation of the data, and 
    returns a list with the lowest and highest values for the axes based on the number of standard deviations
    away from the mean (which can be varied through the second parameter as desired)
    """
    if DiffTS:
        mag = data.std(skipna = True) * n_stddevs
        return [-mag, mag]
    else:
        mean = data.mean(skipna = True)
        std = data.std(skipna = True)
        low = mean - (n_stddevs * std)
        hi = mean + (n_stddevs * std)
        return [low,hi]

def is_number(s):
    """
    A quick helper function pinched from stack overflow to test if a variable is a float or int
    """
    try:
        float(s)
        return True
    except ValueError:
        return False



def filterNoise(df):
    """
    This function applies a scipy.signal.lfilter IIR filter to the time series data to smooth out
    some of the spikes that are visible in this timeseries for each of the columns in the supplied
    pandas dataframe.
    Peters C++ code used a different filter, so this function may be replaced or deprecated in the
    future
    """
    # set up required parameters for the function
    n = 15                 # bigger n means more smoothing
    b = [1.0 / n] * n
    a = 1      
    for col in df.columns:
        df[col] = lfilter(b, a, df[col])
    return df

def calcRollingAvg(df, window = '365D', min_periods = 6):
    """
    The function uses the pandas rolling average function to calculate the yearly rolling
    average of the bands. It returns the dataframe it was given with the column values modified
    by the rolling average function.
    The window size and minimum number of periods for the average to be considered valid are optional
    parameters
    """
    for col in df.columns:
        df[col] = df[col].rolling(window = window, min_periods = min_periods).mean()
    return df

def getSceneFilteredIndex(scene_num, df):
    """
    This function builds a vertical 2D line object with a given x-value.
    This is used draw the lines on the timeseries that represent the scenes being viewed.
    If the scene being viewed is part of that pixel's timeseries, the line returned is a solid line.
    If the scene being viewed is not part of that pixel's timeseries (due to cloud masking), the
    line returned is a dashed line
    """
    # find the date of the scene_number from the list of all dates before clouds mask application
    linedate = pd.to_datetime(dates[scene_num])
    
    # for the simple case, when the scene is part of the pixel's time series
    if linedate in df.index:
        nearest_num = df.index.get_loc(linedate)
        return [nearest_num, '-']
    
    # when the pixel is not part of the time series and is more recent than the most recent valid
    # scene, use the last valid scenes index
    elif linedate > df.index[-1]:
        nearest_num = df.index.get_loc(df.index[-1])
        return [nearest_num, '--']
    
    # when the schene is not part of the pixel's time series, draw the line at the first valid scene's
    # index AFTER the scene in question
    else:   
        nearest_num = df.index.get_loc(df[df.index > linedate].index[0])
        return [nearest_num, '--']

def calcYearDiff(df, years = 1):
    """
    Returns the difference between a timeseries and the timeseries at some point in the future.
    It does this by joining a time adjusted series onto the original data and interpolating any missing
    dates.
    """
    # convert years to days
    days = years * 365
    
    # duplicate data
    df2 = df.copy()

    # reduce the index by a year, essentially moving the average for the following year back one
    # this might need a bit of tweaking to deal with leap years...?
    df2.index = np.array(df2.index) - np.timedelta64(days, 'D')
    
    # building lists of column names that will be needed
    cols = list(df.columns)
    newcols = []
    interpcols = []
    diffcols = []
    for col in cols:
        newcols.append(col + '-1y')
        interpcols.append(col + '-1y_interp')
        diffcols.append(col + '_diff')
    
    # rename df2 columns to avoid clashes in the merge
    df2.columns = newcols

    # SQL style outer join (ie all data is kept, NaNs for blank values)
    df = pd.merge(df, df2, left_index=True, right_index=True, how='outer')
 
    # interpolate the timeseries values in the future and calculate the difference from that date
    for i in range(len(cols)):
        df[interpcols[i]] = df[newcols[i]].interpolate(method='time')
        df[diffcols[i]] = df[interpcols[i]] - df[cols[i]]

    # tidying up unwanted rows and columns
    # can do dropna because if there was no original data, the difference calc result is NaN
    df = df.dropna(subset=[diffcols])
    df = df.drop(cols + newcols + interpcols, axis = 1)
    return df

In [8]:
def getChangePeriodTimeRange(df, daterange):
    """
    This function changes the calculated change date from Peter's algorithm and converts it
    to the equivalent range of row numbers for the given timeseries (supplied as a pandas dataframe).
    The change date calculated is converted to the start and end dates of the quarter, and this is 
    turned into a datetime object, and used in boolean masking to define the first and last row
    of that interval
    """
    
    for i in range(len(daterange)):
        year = str(int(daterange[i]))    
        # generate the string equivalents of the start and end dates for the quarter
        if daterange[i] % 1 == 0.125:
            start = '-01-01'
            end = '-03-31'
        if daterange[i] % 1 == 0.375:
            start = '-04-01'
            end = '-06-30'
        if daterange[i] % 1 == 0.625:
            start = '-07-01'
            end = '-09-30'
        if daterange[i] % 1 == 0.875:
            start = '-10-01'
            end = '-12-31'
        if i == 0:
            startdate = pd.to_datetime(year + start) # turn strings into datetime objects
        else:
            enddate = pd.to_datetime(year + end)    
    
    # calculdate the start and end date index of the range bounded by start and end datetimes
    start_idx = df[(df.index >= startdate) & (df.index <= enddate)].index[0]
    end_idx = df[(df.index >= startdate) & (df.index <= enddate)].index[-1]    
  
    # get the row number of the start and end indexes
    start_row = df.index.get_loc(start_idx)    
    end_row = df.index.get_loc(end_idx)
    
    # return result as a list, ready to directly be passed to axis.set_ylim()
    return [start_row, end_row]

def getChangeDate(xpos, ypos):
    """
    This function returns the max and min change date for a given range of pixels based on the
    output of the algorithm. It works if the range is only a single pixel.
    It returns a string if there was no change detected in that area.
    """
    pix = change[ypos[0]:ypos[1], xpos[0]:xpos[1]]
    pix = pix[pix > 0]
    if len(pix) == 0:
        return "No Change"
    mini = pix.min()
    maxi = pix.max()
    return [mini, maxi]

In [9]:
def drawSpectrum(xpos, ypos, scene_num):
    """
    This function draws the spectral signature for a selected range of pixels in a selected scene.
    It gathers the data together, averages the band values, and returns them plotted on nicely labelled
    axes, which are returned to the caller
    """
    # include Landsat-8 bands incase of future work
    bandorder = {'ultrablue':1, 'blue':2, 'green':3, 'red':4, 'nir':5, 'cirrus':6, 'swir1':7,
                 'swir2':8, 'thermal':9,'panchromatic':10}
    
    # do lots of funkiness to pull the data together and order it so the data are displayed
    # in order of increase wavelength
    xvals = []
    yvals = []
    for band in sat_bands:
        xvals.append(bandorder[band])
        yvals.append(float(data[xpos[0]:xpos[1],ypos[0]:ypos[1],scene_num].sel(band=band).mean(dim=['x','y']).values))
    yvals_sorted = [y for x, y in sorted(zip(xvals,yvals))]
    xlabels_sorted = [y for x, y in sorted(zip(xvals, sat_bands))]
    xvals_sorted = range(len(yvals))
    
    # plot the data, and make it look nice
    plt.plot(xvals_sorted, yvals_sorted)
    ax = plt.gca()
    ax.set_title('Band Values for Selected Area')
    ax.set_ylim([0,6000])
    ax.set_ylabel('Average NBAR Value')
    ax.set_xticks(xvals_sorted)
    ax.set_xticklabels(xlabels_sorted)
    ax.set_xlabel('Band')
    return ax  

In [59]:
def drawScene(scene_num, change_trans, xpos, ypos):
    """
    This function draws a landsat scene from the data in true colour, and overlays the results of the
    change detection algorithm. The scene to be displayed is determined by the scene_num parameter, while the
    transparency (alpha) of the change results is dictacted by the change_trans parameter (0 = transparent,
    1 = totally opaque). It formats the axes appropriately and returns the axes to the caller.
    It also displays the bounding box of the pixels being analysed.
    """
    # colour map included incase of need to display false colour or other in the future
    # could change this to an ordereddict and remove the RGB list created below...?
    colourmap = {'R':'red', 'G':'green', 'B':'blue'}
    
    # define the current colour map to display the change results raster properly
    current_cmap = matplotlib.cm.get_cmap('Reds_r')
    current_cmap.set_under('k', alpha=0.0)
    current_cmap.set_over('r', alpha=1.0)
    current_cmap.set_bad('k', alpha=0.0)  
    
    # combine the data for the 3 bands to be displayed into a single numpy array
    h = data.shape[1]
    w = data.shape[0]
    RGB = ['R','G','B']
    
    # create array to store the RGB info in, and fill by looping through the colourmap variable
    # note the .T at the end, because the data array is setup as a (x,y,t), but imshow works (y,x)
    rawimg = np.zeros((h, w, 3), dtype=np.float32)
    for i in range(len(RGB)):     
        rawimg[:,:,i] = data[:,:,scene_num].sel(band=colourmap[RGB[i]]).T
        
    # equalizing for all bands together 
    img_toshow = exposure.equalize_hist(rawimg, mask = np.isfinite(rawimg))    

    # displaying the results and formatting the axes etc
    plt.imshow(img_toshow)
    ax = plt.gca()
    ax.set_title('True Colour Landsat Scene, taken\n' + dates[scene_num] + ', over ' + location)
    ax.imshow(change_flat_mask, alpha = change_trans, interpolation='none', cmap = current_cmap, clim = [0.5, 0.6])
    
    # plot up the displayed pixel
    height = abs(ypos[1] - ypos[0])
    width = abs(xpos[1] - xpos[0])
    if height > 2 and width > 2:
        rect = matplotlib.patches.Rectangle((xpos[0],ypos[0]), abs(xpos[1] - xpos[0]), abs(ypos[1] - ypos[0]),
                                            color = 'blue', linestyle = '-', fill = False, alpha = change_trans)
        ax.add_patch(rect)
    else:
        ax.plot((xpos[0] + xpos[1])/2, (ypos[0] + ypos[1])/2, color = 'b', marker = '.', alpha = change_trans)
    return ax

In [25]:
def drawAllSubplots(left_scene_num, right_scene_num, change_trans, ts_bands, xpos, ypos, BBoxWidth, BBoxHeight):
    """
    This function defines what plots will be shown, and in what order. This function is called
    by the the analyse function and also during the onclick event.
    It returns nothing, but draws the axes on command
    """
    # need to do more thinking about how to make this more consistent and logical with list index selection
    # ie the end of the range is not included, so might need to add one
    if BBoxWidth == 1:
        xrange = [xpos, xpos + 1]
    else:
        xrange = [int(xpos - BBoxWidth/2), int(xpos + BBoxWidth/2)]
    if BBoxHeight == 1:
        yrange = [ypos, ypos + 1]
    else:
        yrange = [int(ypos - BBoxHeight/2), int(ypos + BBoxHeight/2)]
    
    # might need some changing in the future regarding formating, plot spacing etc.   
    ax1 = plt.subplot2grid([7,4],[0,0], rowspan = 2, colspan = 2)
    ax1.clear()
    ax1 = drawScene(left_scene_num, change_trans, xrange, yrange)    

    ax2 = plt.subplot2grid([7,4],[0,2], rowspan = 2, colspan = 2)
    ax2.clear()
    ax2 = drawScene(right_scene_num, change_trans, xrange, yrange)  

    ax3 = plt.subplot2grid([7,4],[2,0], rowspan = 2, colspan = 4)
    ax3.clear()
    ax3 = drawTimeSeries(xrange, yrange, ts_bands, left_scene_num, right_scene_num)

    ax4 = plt.subplot2grid([7,4],[4,0], rowspan = 2, colspan = 2)
    ax4.clear()
    ax4 = drawScatter(left_scene_num, ts_bands, xrange, yrange)    

    ax5 = plt.subplot2grid([7,4],[4,2], rowspan = 2, colspan = 2)
    ax5.clear()
    ax5 = drawScatter(right_scene_num, ts_bands, xrange, yrange) 
    
    ax6 = plt.subplot2grid([7,4],[6,0], rowspan = 1, colspan = 2)        
    ax6.clear()
    ax6 = drawSpectrum(xrange, yrange, left_scene_num)                       

    ax7 = plt.subplot2grid([7,4],[6,2], rowspan = 1, colspan = 2)
    ax7.clear()
    ax7 = drawSpectrum(xrange, yrange, right_scene_num)

    plt.tight_layout()
    plt.draw()

### The main function

In [37]:
# from matplotlib.widgets import RetangleSelector 
global xpos
global ypos
xpos = 0
ypos = 0
  
def analysis(left_scene_num, right_scene_num, change_trans, ts_bands, BBoxWidth = 1, BBoxHeight = 1):

    def onclick(event):
        global xpos
        global ypos
        xpos = int(event.xdata)
        ypos = int(event.ydata)
        drawAllSubplots(left_scene_num, right_scene_num, change_trans, ts_bands, xpos, ypos, BBoxWidth, BBoxHeight)
       
    fig = plt.figure(figsize=[10,15])
    plt.subplots_adjust(hspace = 0.6)
    drawAllSubplots(left_scene_num, right_scene_num, change_trans, ts_bands, xpos, ypos, BBoxWidth, BBoxHeight)
    cid = fig.canvas.mpl_connect('button_press_event', onclick)

In [39]:
def drawScatter(scene_num, ts_bands, xpos, ypos):
    """
    This function draws a 2D scatter plot of the scene, based on the two bands of interest (specified by
    x_axis and y_axis). It returns the scatter plot to the caller
    """
    x_axis = ts_bands[0]['band']
    y_axis = ts_bands[1]['band']
    
    # get the 1D array of the values for each axis
    x = data[:,:,scene_num].sel(band = x_axis).values
    y = data[:,:,scene_num].sel(band = y_axis).values
    
    sel_x = data[xpos[0]:xpos[1], ypos[0]:ypos[1], scene_num].sel(band = x_axis).values
    sel_x_all = np.full(x.shape, np.nan)
    sel_x_all[xpos[0]:xpos[1],ypos[0]:ypos[1]] = sel_x
    
    sel_y = data[xpos[0]:xpos[1], ypos[0]:ypos[1], scene_num].sel(band = y_axis).values
    sel_y_all = np.zeros_like(y)
    sel_y_all[xpos[0]:xpos[1],ypos[0]:ypos[1]] = sel_y
    
    
    # build a mask of where the values are both valid (ie not NaN)
    mask = np.isfinite(x) & np.isfinite(y)
    mask2 = np.isfinite(sel_x_all) * np.isfinite(sel_y_all)
    
    # make plot, label axes and return
    plt.plot(x[mask], y[mask],'.', color = 'blue', label = 'All Points')
    plt.plot(sel_x_all[mask2], sel_y_all[mask2],'.', color = 'red', label = 'Selected Points')
    ax = plt.gca()
    ax.set_xlabel(x_axis)
    ax.set_xlim([0,1])
    ax.set_ylabel(y_axis)
    ax.set_ylim([0,5])
    ax.legend()
    return ax

In [53]:
def drawTimeSeries(xpos,ypos,ts_bands,left_scene_num = 0, right_scene_num = data.shape[2]-1):
    """
    This function draws the time series for a given pixel and band (or list of 2 bands).
    It calls the timeSeries function to gather the data and strip out the cloud, and then
    formats the axes appropriately for up to 2 bands and returns them to the caller
    """ 
    # get the data, filter cloud, filter noise, calculate rolling average, drop NaN rows, calc difference
    df = gatherAndPrepTSData(xpos, ypos, ts_bands)
    yearDiff = True

    # see if/when that pixel changed
    changedate = getChangeDate(xpos, ypos)
    changed = False
    if isinstance(changedate, list):
        changed = True
        # if it did change, get the row numbers of the first and last scene taken during the change quarter
        # to draw the box aronud the potential change period
        changetimerange = getChangePeriodTimeRange(df, changedate)
    
    # only plot up the first 4 bands requested (this might be changed soon)
    cols = list(df.columns)
    colours = ['green', 'red', 'blue', 'orange', 'black']
    axs = []
    n_axes = min(len(cols),5)

    leftline = getSceneFilteredIndex(left_scene_num, df)    
    rightline = getSceneFilteredIndex(right_scene_num, df)    
    albedo_thresh = 0.04
    evi_thresh = -0.05
    resid_evi = 0.18    
    
    plt.plot(range(len(df)), df[cols[0]], color=colours[0], label = cols[0])
    ax0 = plt.gca()
    pos1 = ax0.get_position()
    pos2 = pos1
#     pos2 = [pos1.x0 + (0.03 * n_axes), pos1.y0,  pos1.width / (1 + (0.1 * (n_axes - 1))), pos1.height] 
    ax0.set_position(pos2)
    ax0.spines['right'].set_color(colours[0])

    ax0.axvline(x = leftline[0], linestyle = leftline[1]) 
    ax0.axvline(x = rightline[0], linestyle = rightline[1])
    ax0.axhline(y = 0, color = 'black', linestyle = '-')
            
    if changed:
        ax0.axvspan(changetimerange[0],changetimerange[1], color = 'gold', alpha = 0.7)
#         yrange = define_TS_YAxisRange(df[cols[0]], yearDiff, 2)
#         ax0.text(changetimerange[1], yrange[1] - ((yrange[1] - yrange[0]) / 10), changedate)
        
    ax_date = ax0.twiny()
    ax_date.plot(df.index, df[cols[0]], alpha = 0)
    ax_date.set_position(pos2)
    readable_coords = 'x = ' + str(xpos[0]) + ':' + str(xpos[1]) + ', y = ' + str(ypos[0]) + ':' + str(ypos[1])
    ax0.set_title('Time Series values for pixels ' + readable_coords, y = 0)

    for i in range(1, n_axes):
        axn = ax0.twinx()
        axn.set_position(pos2)
        axn.spines['right'].set_position(('axes', - (0.1 * i)))
        axn.spines['right'].set_color(colours[i])
        axn.plot(range(len(df)), df[cols[i]], color=colours[i], label = cols[i])
        axs.append(axn)
    
    axs = [ax0] + axs
    
    for i, ax in enumerate(axs):
        yrange = define_TS_YAxisRange(df[cols[i]], ts_bands[i]['TSDiff'], 2)
        ax.set_ylim(yrange)
        ax.set_xlim([0, len(df)])
        if i == 0:
            ax.set_ylabel(cols[i], color=colours[i], labelpad = -10)
            ax.axhline(y = evi_thresh, color = colours[i], linestyle = '--')
            after_evi = float(data[xpos[0]:xpos[1], ypos[0]:ypos[1], right_scene_num].sel(band = 'evi').mean().values)
            ax0.text(rightline[0], yrange[1] - ((yrange[1] - yrange[0]) / 10), 'EVI = ' + str(round(after_evi, 3)))       
        else:
            ax.set_ylabel(cols[i], color=colours[i], labelpad = -40)
        if i == 1:
            ax.axhline(y = albedo_thresh, color = colours[i], linestyle = '--')
        if i == 2:
            ax.axhline(y = resid_evi, color = colours[i], linestyle = '--')
    return axs

### Executing the function

In [57]:
% matplotlib notebook

ts_bands = [{'name':'evi_diff', 'band':'evi', 'cloud_mask':True, 'smoothed':True, 'RollingAve':True, 'TSDiff':True},
{'name':'albedo_diff', 'band':'albedo', 'cloud_mask':True, 'smoothed':True, 'RollingAve':True, 'TSDiff':True},
{'name':'evi_raw', 'band':'evi', 'cloud_mask':True, 'smoothed':True, 'RollingAve':False, 'TSDiff':False}]


with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    interact(analysis,
             left_scene_num = IntSlider(value = 1, min = 0, max = num_scenes -1,
                                        description = "Left Scene Number"),
             right_scene_num = IntSlider(value = num_scenes -1, min = 0, max = num_scenes -1,
                                         description = "Right Scene Number"),
             change_trans = FloatSlider(value = 0.6, min = 0, max = 1, description = "Change Mask Transparency"),
#              ts_bands = fixed(['evi','albedo']),
             ts_bands = fixed(ts_bands),
            BBoxWidth = IntSlider(value = 1, min = 1, max = x, description = "BBox Width"),
            BBoxHeight = IntSlider(value = 1, min = 1, max = y, description = "BBox Height"))

interactive(children=(IntSlider(value=1, description='Left Scene Number', max=671), IntSlider(value=671, description='Right Scene Number', max=671), FloatSlider(value=0.6, description='Change Mask Transparency', max=1.0), IntSlider(value=1, description='BBox Width', max=171, min=1), IntSlider(value=1, description='BBox Height', max=143, min=1), Output()), _dom_classes=('widget-interact',))

In [66]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio import crs

change_rio = rasterio.open(location + 'change_time.img')

change_rio.index(xpos, ypos)

(-153777, -24908)

In [60]:
xpos

118